In [111]:
from typing import Union, List, Tuple
import numpy as np

In [112]:
with open('input.txt') as f:
    n = int(f.readline()) # количество задач
    task_rate = [int(task) for task in f.readline().split()] #N целых чисел от 1 до 4 категорий сложности задач
    estimated_task_time = [float(estimate) for estimate in f.readline().split()] # N вещественных положительных чисел оценочного времени для задач
    slaves_count = int(f.readline()) # целое число M, количество разработчиков
    slaves_task_coeffs = [list(map(float, f.readline().split())) for _ in range(slaves_count)] # M строк содержат по 4 вещественных положительных числа — коэффициенты каждого разработчика.

In [113]:
class GeneticAlgorithm:
    rng = np.random.default_rng()
    def __init__(
        self,
        slaves_count: int,
        estimated_task_time: Union[List[float], np.ndarray],
        task_rate: Union[List[int], np.ndarray],
        slaves_task_coeffs: Union[List[List[float]], np.ndarray],
        population_size: int = 100,
        n_offspring: int = 2,
        n_crossover: int = 3,
        mutation_probability: int = 0.90,
    ):
        self.slaves_count = slaves_count
        self.n_offspring = n_offspring
        self.n_crossover = n_crossover
        self.mutation_probability = mutation_probability
        self.population_size = population_size
        self.slaves_task_coeffs = np.array(slaves_task_coeffs)
        self.task_rate = np.array(task_rate)
        self.estimated_task_time = np.array(estimated_task_time)
        # calculation
        self.slaves_indexes = np.arange(self.slaves_count)
        self.task_count = len(self.task_rate)
        self.slaves_time_on_each_task = self.map_task_time()
        self._population = self.create_population()
        self._fitness = self.fitness()
        self.best_project_time, self.best_distribution = self.get_best_result()

    def map_task_time(self) -> np.ndarray:
        return self.estimated_task_time * [
            np.array([coeff[task - 1] for task in self.task_rate]) for coeff in slaves_task_coeffs
        ]
    def create_population(self) -> np.ndarray:
        return np.array(
            [GeneticAlgorithm.rng.choice(self.slaves_indexes, size=self.task_count) for _ in range(self.population_size)]
        )

    def get_best_result(self) -> Tuple[float, np.ndarray]:
        return np.max(self._fitness), self._population[np.argmax(self._fitness)]

    
    def crossover(self) -> None:
        def single_point_crossover(a: np.ndarray, b: np.ndarray, point: int) -> Tuple[np.ndarray, np.ndarray]:
            return (np.hstack((a[:point], b[point:])), np.hstack((b[:point], a[point:])))

        self.new = np.zeros((self.population_size - self._population.shape[0], self.task_count), dtype=int)
        for i in range(len(self.new)):
            a, b = self.rng.choice(self._population.shape[0], size=2, replace=False)
            point = self.rng.integers(low=0, high=self.task_count)
            a_, b_ = single_point_crossover(self._population[a], self._population[b], point)
            self.new[i] = self.rng.choice([a_, b_], size=1)

    def mutation(self) -> None:
        p = GeneticAlgorithm.rng.random(size=self.new.shape[0]) < self.mutation_probability
        for i, p in zip(np.arange(self.new.shape[0]), p):
            if p:
                slave = GeneticAlgorithm.rng.choice(self.slaves_indexes, size=2)
                task = GeneticAlgorithm.rng.integers(low=0, high=self.task_count - 1, size=2)
                self.new[i, task] = slave
    
    def selection(self) -> np.ndarray:
        new_idxs = np.argsort(self._fitness)[-self.n_offspring :]
        self._population = self._population[new_idxs]
        self._fitness = self._fitness[new_idxs]
        return new_idxs

    def run(self) -> None:
        self.selection()
        self.crossover()
        self.mutation()
        self._population = np.concatenate([self._population, self.new], axis=0)
        self._fitness = self.fitness()
        time, dist = self.get_best_result()
        if time > self.best_project_time:
            self.best_project_time, self.best_distribution = time, dist

    def fitness(self) -> np.ndarray:
        result = np.zeros(self.population_size)
        for individual in range(self.population_size):
            individual_result = np.zeros(self.slaves_count)
            for slave in self.slaves_indexes:
                task_indexes = np.where(self._population[individual] == slave)[0]
                individual_result[slave] = np.sum(self.slaves_time_on_each_task[slave, task_indexes])
            result[individual] = 10 ** 6 / np.max(individual_result)
        return result
    

In [114]:
ga = GeneticAlgorithm(slaves_count, estimated_task_time, task_rate, slaves_task_coeffs)

In [115]:
for i in range(100):
  ga.run()

In [116]:
with open('output.txt', 'w') as f:
    f.write(' '.join(map(str, ga.best_distribution + 1)))
    